#### Cargamos las funciones y archivos necesarios

In [1]:
#The following packages are required to read and write files, as well as generate linear algebra calculations
using DelimitedFiles
using LinearAlgebra

#The following files with Julia functions are required
include("Operaciones_Basicas.jl")
include("Red_Dual.jl")
include("Separacion_Arreglos_X_Y.jl")
include("Generador_Vecindades_Pto_Arbitrario.jl")
include("Algoritmo_Voronoi.jl")
include("Algoritmo_Cluster_Principal.jl")
include("Diccionario_Indices_Centroides_Voronoi.jl")

diccionario_Centroides_Indice_Voronoi (generic function with 1 method)

## Algoritmo Radio Cluster (Funciona para todo SL, pero requiere conocer el beta)

In [4]:
NSides = 5; #Rotational symmetry of the quasiperiodic lattice
Star_Vectors = [[BigFloat(1),0]]; #Arrangement that will contain the star vectors
for i in 1:(NSides-1)
    push!(Star_Vectors, [cos((2*i)*pi/NSides), sin((2*i)*pi/NSides)]); #Vertices of the polygon with "NSides" sides
end
Alphas_Array = fill(0.0, NSides); #Array with the alpha constants of the GDM
Average_Distance_Stripes = fill(NSides/2, NSides); #Array with the average spacing between stripes

Error_Margin = 300; #Margin of error in the integers "n" associated with the star vectors of the GDM
Radius = 720; #Radius of the main cluster patch for the neighborhood around an arbitrary point

#Information for the N(R) array calculation
Step = 100000;
R = (Radius/Step):(Radius/Step):Radius;

SL = 1e12; #Size of the half side of the square centered on the (0,0) where a starting position is sought.
###################################################################################################################

for k in 1:400
    open("Estado_TorquatoNR_N$(NSides)_Alfa0P0_1.txt", "w") do file
        write(file, "Estamos generando el cluster $(k).")
    end

    APoint = punto_Arbitrario(SL); #We generate an arbitrary point inside the square centered on the (0,0)

    #We generate the local neighborhood around the Test_Point point
    Dual_Points = region_Local_Voronoi(Error_Margin, Average_Distance_Stripes, Star_Vectors, Alphas_Array, APoint);
    unique!(Dual_Points);

    Cluster_Sites = [];
    for e in Dual_Points
        if norm(e-APoint) < Radius
            push!(Cluster_Sites, e)
        end
    end
    
    #Sort the list by the distance to the center of the cluster
    sort!(Cluster_Sites, by=x->norm(x-APoint))

    #Generate the array that will be the N(r) array
    N_R = zeros(length(R));
    
    #We obtain the first element that satisfies that his distant to the center of the cluster is bigger than the R[i]
    FirstIndex = 1;
    for i in 1:length(R)
        for j in FirstIndex:length(Cluster_Sites)
            if j == length(Cluster_Sites)
                N_R[i] = j; First_Index = j;
            else
                (norm(Cluster_Sites[j]-APoint) > R[i]) ? (N_R[i] = j-1; FirstIndex = j; break) : (nothing)
            end
        end
    end
    
    writedlm("TorquatoNR_N$(NSides)_Alfa0P0_1_$(k).csv", N_R, ',')
end